In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
import torch
import pandas as pd
from torchmetrics.text import ROUGEScore
from transformers import BartForConditionalGeneration, BartTokenizerFast
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv("../../data/labelled/reviews/splits/test.csv")

In [3]:
df.head(2)

,input_reviews,label_reviews
0,Below are the reviews of a product: \n- I purc...,pros:\n- Reduces glare\n- Very impressive\n- W...
1,Below are the reviews of a product: \n- Well w...,pros:\n- Huge capacity\n- Beautiful design\n- ...


In [4]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device

'cuda'

In [5]:
model_path = "saichandrapandraju/bart-summarization-amazon-product-reviews"

In [6]:
model = BartForConditionalGeneration.from_pretrained(model_path)

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

C:\Users\u.hoitash\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\u.hoitash\.cache\huggingface\hub\models--saichandrapandraju--bart-summarization-amazon-product-reviews. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

In [7]:
%%capture
model.to(device)

In [8]:
tokenizer = BartTokenizerFast.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [9]:
rouge = ROUGEScore()

In [10]:
batch_size = 8

In [11]:
rouge_scores = {"rouge1":[], "rouge2":[], "rougeL": []}
preds_all = []
for idx in tqdm(range(0, len(df), batch_size)):
    batch = df[idx:idx+batch_size]
    product_reviews = batch.input_reviews.to_list()
    actual_summary = batch.label_reviews.to_list()

    input_ids = tokenizer(product_reviews, max_length=512, return_tensors="pt", padding=True, truncation=True)["input_ids"].to(device)
    summary_ids = model.generate(input_ids, min_length=0, max_length=256)
    preds = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    preds_all.extend(preds)

    for pred, target in zip(preds, actual_summary):
        rouge_all = rouge(pred, target)
        rouge_scores['rouge1'].append(rouge_all['rouge1_fmeasure'].item())
        rouge_scores['rouge2'].append(rouge_all['rouge2_fmeasure'].item())
        rouge_scores['rougeL'].append(rouge_all['rougeL_fmeasure'].item())


  0%|          | 0/2500 [00:00<?, ?it/s]

In [12]:
df['rouge1'] = rouge_scores['rouge1']
df['rouge2'] = rouge_scores['rouge2']
df['rougeL'] = rouge_scores['rougeL']
df['bart_large_finetuned'] = preds_all

In [13]:
df.to_csv("bart_large_reviews_scores.csv", index=False)

In [14]:
df = pd.read_csv("bart_large_reviews_scores.csv")

In [14]:
df.head()

,input_reviews,label_reviews,rouge1,rouge2,rougeL,bart_large_finetuned
0,Below are the reviews of a product: \n- I purc...,pros:\n- Reduces glare\n- Very impressive\n- W...,0.372340,0.107527,0.244681,pros:\n- No glare in photos\n- Well-built\n- W...
1,Below are the reviews of a product: \n- Well w...,pros:\n- Huge capacity\n- Beautiful design\n- ...,0.333333,0.132653,0.232323,pros:\n- Charges devices at almost full oem sp...
2,Below are the reviews of a product: \n- Charge...,pros:\n- Charges fast\n- Works well\n- Charges...,0.397059,0.164179,0.294118,pros:\n- Charges fast\n- Perfect with travel c...
3,Below are the reviews of a product: \n- Given ...,pros:\n- Very rugged and shock resistant\n- Pr...,0.521739,0.231076,0.316206,pros:\n- Rugged and shock resistant\n- Easy to...
4,Below are the reviews of a product: \n- I orde...,pros:\n- Sturdy packaging and build (stand up ...,0.404669,0.086275,0.202335,pros:\n- Sturdy angled plug with a sturdy cord...


In [15]:
scores = {"rouge1": df.rouge1.mean(), "rouge2": df.rouge2.mean(), "rougeL": df.rougeL.mean()}

In [16]:
scores

{'rouge1': 0.4472946444006637,
 'rouge2': 0.20606587370915805,
 'rougeL': 0.3178484115054831}